In [ ]:
from EyeDetection import EyeDetector
from Preprocessing import crop_and_resize_eye

import cv2 as cv
import mediapipe as mp
import numpy as np
from datetime import datetime
import os

THRESHOLD = 3

ED = EyeDetector()

# 중앙 눈꺼풀 포인트 인덱스 (Mediapipe 기준)
RIGHT_CENTER_UPPER = 159
RIGHT_CENTER_LOWER = 145
LEFT_CENTER_UPPER = 386
LEFT_CENTER_LOWER = 374

# 오른쪽 눈 (화면 기준 왼쪽)
RIGHT_EYE_ALL = ED.RIGHT_EYE_IDX + [145, 144, 153, 154]

# 왼쪽 눈 (화면 기준 오른쪽)
LEFT_EYE_ALL = ED.LEFT_EYE_IDX+[374, 373, 380, 381]

# ---------------------------
# 눈 감김 여부 판단 함수
# ---------------------------
def is_eye_closed_by_center(landmarks, upper_idx, lower_idx, ih, iw, threshold=THRESHOLD):
    try:
        upper_y = landmarks[upper_idx].y * ih
        lower_y = landmarks[lower_idx].y * ih
        distance = abs(lower_y - upper_y)
        return distance < threshold  # True = 감김, False = 열림
    except:
        return None

# ---------------------------
# 랜드마크 좌표 찍기 함수
# ---------------------------
def draw_eye_landmark_points(image, landmarks, indices, color=(255, 0, 0)):
    ih, iw = image.shape[:2]
    for idx in indices:
        if idx < len(landmarks):
            x = int(landmarks[idx].x * iw)
            y = int(landmarks[idx].y * ih)
            cv.circle(image, (x, y), 2, color, -1)

# 웹캠 시작
cap = cv.VideoCapture(0)
print("'q' 키를 누르면 종료됩니다.")

left_eye_img = None
right_eye_img = None

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
    ih, iw = frame.shape[:2]
    results = ED.face_mesh.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark

        # 눈 감김 판단
        left_closed = is_eye_closed_by_center(landmarks, LEFT_CENTER_UPPER, LEFT_CENTER_LOWER, ih, iw)
        right_closed = is_eye_closed_by_center(landmarks, RIGHT_CENTER_UPPER, RIGHT_CENTER_LOWER, ih, iw)
        
        # 결과 텍스트 표시
        if left_closed is not None:
            cv.putText(frame, f"Left Eye: {'Closed' if left_closed else 'Open'}",
                       (30, 50), cv.FONT_HERSHEY_SIMPLEX, 1,
                       (0, 0, 255) if left_closed else (0, 255, 0), 2)

        if right_closed is not None:
            cv.putText(frame, f"Right Eye: {'Closed' if right_closed else 'Open'}",
                       (30, 100), cv.FONT_HERSHEY_SIMPLEX, 1,
                       (0, 0, 255) if right_closed else (0, 255, 0), 2)

        # 눈 랜드마크 점 찍기 (추가된 기능)
        draw_eye_landmark_points(frame, landmarks, RIGHT_EYE_ALL, color=(255, 150, 0))  # 파란색
        draw_eye_landmark_points(frame, landmarks, LEFT_EYE_ALL, color=(255, 150, 0))   # 파란색

    
    # RGB 변환
    L, R = ED.get_eye_bounding_boxes(frame)
    right_eye_img = crop_and_resize_eye(frame, L)
    left_eye_img = crop_and_resize_eye(frame, R)

    if left_eye_img is not None:
        cv.imshow("Left Eye", left_eye_img)
    if right_eye_img is not None:
        cv.imshow("Right Eye", right_eye_img)

    # 원본 영상 표시
    cv.imshow("Webcam", frame)
    key = cv.waitKey(1) & 0xFF

    # 종료: 'q' 키
    if key == ord('q'):
        print("종료합니다.")
        break

# 종료 처리
cap.release()
cv.destroyAllWindows()